In [1]:
import numpy as np
from itertools import count
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import csv 
from scipy import signal
from scipy.signal import butter, filtfilt
import os
import matplotlib as mpl
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider
from scipy.signal import freqs

mpl.rcParams['figure.dpi']= 150

In [2]:
def lowpass_filter(dataframe, f_c, f_s):
    
    n = 5 # order of the filter
    b, a = butter(n, f_c, fs=f_s, btype='low', analog=False)
    
    return filtfilt(b, a, dataframe)

In [3]:
file_path = '/Users/thomas/Documents/gwks/pico/flow_data_analysis/flow_data/about_60/about_60_hz3.csv'

df = pd.read_csv(file_path).iloc[50:-50]
df['time'] = df.index
df['sensor'] = df['Timestamp (unix time nano seconds)'].str.replace(' ', '')
df = df[['time', 'sensor']]
df = df.reset_index(drop=True)

df['time'] = df['time'].apply(lambda x: int(x))

display(df.head())

,time,sensor
0,1718375494399105000,Hall
1,1718375494416037000,Hall
2,1718375494433124000,Hall
3,1718375494449987000,Hall
4,1718375494466875000,Hall


In [4]:
hall_times = list(df[df.sensor=='Hall'].time)

for i in range(len(hall_times)-1):
    frequency = 1/(hall_times[i+1]-hall_times[i])*1e9
    if i==0: 
        df.loc[df.time>=0, 'hall_freq'] = frequency/2
        df.loc[df.time>=0, 'hall_GPM'] = frequency/2 #/7.5 * 0.264172
    df.loc[df.time>=hall_times[i], 'hall_freq'] = frequency/2
    df.loc[df.time>=hall_times[i], 'hall_GPM'] = frequency/2 #/7.5 * 0.264172

timestamps, values = list(df.time), list(df.hall_GPM)

def cool_plot(target_fs, fc):

    new_timestamps = np.linspace(min(timestamps), max(timestamps), int((max(timestamps) - min(timestamps))/10**9 * target_fs))
    interp_func = interp1d(timestamps, values, kind='linear')
    interpolated_values = interp_func(new_timestamps)

    try:
        filtered = lowpass_filter(interpolated_values, fc, target_fs)

        fig, ax = plt.subplots(1,2, figsize=(14, 6))

        ax[0].plot(df.time, df.hall_GPM, color='black', alpha=0.2)
        ax[0].plot(new_timestamps, interpolated_values, '-o', alpha=0.2)
        ax[0].plot(new_timestamps, filtered, color='red', alpha=0.9)

        point = list(range(0, int(max(df.hall_GPM)+5), 1))
        vals = [1/np.sqrt(1+(x/fc)**(2*5)) for x in point]
        ax[1].plot(point, vals)
        ax[1].set_ylim([0,1])
        
        plt.show()
    except: 
        print("ERROR: You need to keep f_c < f_s/2 (Nyquist frequency)")

# Define the sliders
cutoff_slider = FloatSlider(min=0.1, max=max(df.hall_GPM)+5, step=0.1, value=300, description='f_cutoff')
sampling_slider = FloatSlider(min=1, max=500, step=1, value=1000, description='f_sampling')

# Create the plot
interact(cool_plot, fc=cutoff_slider, target_fs=sampling_slider);

interactive(children=(FloatSlider(value=500.0, description='f_sampling', max=500.0, min=1.0, step=1.0), FloatS…

In [5]:
file_path = '/Users/thomas/Documents/gwks/pico/flow_data_analysis/flow_data/about_60/about_60_hz3.csv'

df = pd.read_csv(file_path).iloc[50:-50]
df['time'] = df.index
df['sensor'] = df['Timestamp (unix time nano seconds)'].str.replace(' ', '')
df = df[['time', 'sensor']]
df = df.reset_index(drop=True)

df['time'] = df['time'].apply(lambda x: int(x))

#df = df[220:280]

display(df.head())

,time,sensor
0,1718375494399105000,Hall
1,1718375494416037000,Hall
2,1718375494433124000,Hall
3,1718375494449987000,Hall
4,1718375494466875000,Hall


In [6]:
# Get the plot data
def plot_data(alpha, f_c, f_s):

    hall_times = list(df[df.sensor=='Hall'].time)

    for i in range(len(hall_times)-1):
        frequency = 1/(hall_times[i+1]-hall_times[i])*1e9
        if i==0: 
            df.loc[df.time>=0, 'hall_freq'] = frequency/2
            df.loc[df.time>=0, 'hall_GPM'] = frequency/2 #/7.5 * 0.264172
        df.loc[df.time>=hall_times[i], 'hall_freq'] = frequency/2
        df.loc[df.time>=hall_times[i], 'hall_GPM'] = frequency/2 #/7.5 * 0.264172

    # Low pass filer
    df['hall_GPM_bias_filtered'] = lowpass_filter(df['hall_GPM'], f_c, f_s)

    # Exponential weighted average
    W = [0]*len(df)
    theta = list(df.hall_GPM)
    for t in range(len(df)-1):
        W[t+1] = alpha*W[t] + (1-alpha)*theta[t+1]
    df['hall_GPM_bias_expWA'] = [np.nan]*10 + W[10:]

    # Get rid of outliers
    upper_limit = 1000
    df.loc[df['hall_GPM'] > upper_limit, 'hall_GPM'] = np.nan
    df.loc[df['hall_GPM_bias_filtered'] > upper_limit, 'hall_GPM_bias_filtered'] = np.nan
    df.loc[df['hall_GPM_bias_expWA'] > upper_limit, 'hall_GPM_bias_expWA'] = np.nan
    df['hall_GPM'] = df['hall_GPM'].mask(df['hall_GPM'].isnull()).ffill()
    df['hall_GPM_bias_filtered'] = df['hall_GPM_bias_filtered'].mask(df['hall_GPM_bias_filtered'].isnull()).ffill()
    df['hall_GPM_bias_expWA'] = df['hall_GPM_bias_expWA'].mask(df['hall_GPM_bias_expWA'].isnull()).ffill()

    return list(df.hall_GPM), list(df.hall_GPM_bias_filtered), list(df.hall_GPM_bias_expWA)

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider

def plot_interactive(alpha, f_c, f_s):
    
    fig, ax = plt.subplots(1,2, figsize=(14, 6), sharey=True)

    GPM, GPM_filtered, GPM_expWA = plot_data(alpha, f_c, f_s)
    
    ax[0].plot(GPM[100:], alpha=0.15, label='Original signal', color='black')
    ax[0].plot(GPM_expWA[100:], alpha=0.7, label='Exponential weighted average', color='tab:blue')

    ax[1].plot(GPM[100:], alpha=0.15, label='Original signal', color='black')
    ax[1].plot(GPM_filtered[100:], alpha=0.7, label='Low pass filter', color='tab:orange')

    ax[0].set_ylabel('Frequency')
    ax[0].set_xlabel('Time')
    ax[0].legend()
    ax[1].legend()
    #plt.ylim([3,11])
    plt.show()

# Define the sliders
alpha_slider = FloatSlider(min=0.01, max=0.99, step=0.01, value=0.93, description='Alpha')
cutoff_slider = FloatSlider(min=0.1, max=0.49, step=0.01, value=0.16, description='f_cutoff')
sampling_slider = FloatSlider(min=1, max=40, step=0.1, value=20, description='f_sampling')

# Create the plot
interact(plot_interactive, alpha=alpha_slider, f_c=cutoff_slider, f_s=sampling_slider);

interactive(children=(FloatSlider(value=0.93, description='Alpha', max=0.99, min=0.01, step=0.01), FloatSlider…